In [1]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [2]:
video_folder = "/home/persie/Videos/mokap/"

In [3]:
session_id = os.listdir(video_folder)
video_file = []
for folder in session_id:
    files = os.listdir(os.path.join(video_folder, folder))
    for file in files:
        video_file.append(os.path.join(video_folder, folder, file))
    
#session_number


In [4]:
#cap = cv.VideoCapture(video_file[33])
cap = cv.VideoCapture("/home/persie/Videos/mokap/240809-1240/240809-1240_cam2_banana_session23.mp4")

## Find frame for background subtraction ##
### Find Confident Background ###

In [ ]:
# get total number of frames
total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
sample_rate = 100
success, frame = cap.retrieve() # get the next frame
if success:
    frame_array = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
fno = 0
print(success)
while success:
    if fno % sample_rate == 0:
        success, frame = cap.retrieve()
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame_array = np.dstack((frame_array,gray))

    # read next frame
    success = cap.grab()
    fno += 1

    

In [ ]:
print(total_frames)
for f in range(0, frame_array.shape[2], 1):
    cv.imshow("image", frame_array[:,:,f])
    cv.waitKey(300)
    if cv.waitKey(20) & 0xFF == ord('q'):
        break
cv.destroyAllWindows()

### Find Confident Foreground ###

In [7]:
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
# ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
blur = cv.GaussianBlur(gray,(7,7),0)
ret3,thresh = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

In [8]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)

# sure background area
#sure_bg = cv.dilate(opening,kernel,iterations=3)
sure_fg = 255 - cv.morphologyEx(opening,cv.MORPH_CLOSE,np.ones((6,6),np.uint8), iterations = 2)
 
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,3)
ret, sure_bg = cv.threshold(dist_transform,0.8*dist_transform.max(),255,0)
 
# Finding unknown region
sure_bg = np.uint8(255-sure_bg)
unknown = cv.subtract(sure_bg,sure_fg)

In [9]:
# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
 
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
 
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [10]:
markers = cv.watershed(frame,markers)
frame[markers == -1] = [255,0,0]

In [11]:
out_frame = thresh

while True:
    #ret, out_frame = cap.read()
    cv.imshow("Original", frame)
    cv.imshow("Fg", sure_fg)
    cv.imshow("bg", sure_bg)
    cv.imshow("unknown", unknown)

    if cv.waitKey(20) & 0xFF == ord('q'):
        break
        
cv.destroyAllWindows()